In [1]:
import pandas as pd
import re

In [2]:
def predicting_txt_to_dataframe(path):    
    with open(path, 'r') as f:
        txt = str(f.read())
        txt_list = txt.split('\n')
    row_list = list()
    tmp_list = list()
    for row in txt_list:
        if row == '--------------------':
            tmp_list[0] = tmp_list[0].replace('article_id:', '')
            tmp_list[0] = int(tmp_list[0])
            row_list.append(tmp_list)
            tmp_list = list()
            continue
        if len(row) > 0:
            tmp_list.append(row)
    df = pd.DataFrame(row_list, columns=['article_id','text'])
    return df

In [3]:
df = predicting_txt_to_dataframe('/home/Danny/AI-CUP-2020/datasets/stage5/test.txt')
df

,article_id,text
0,0,醫師：最近人有沒有什麼不舒服？民眾：沒有。醫師：沒有喔。我們本來說要月，年底才打對不對？民眾...
1,1,醫師：要去上課嗎？民眾：沒有。這學期已經開始不上課了。醫師：啊今天的，呃，應該是昨天的抽血。...
2,2,醫師：後來那個吃藥沒有問題對不對？民眾：沒有。醫師：诶？你是打針的？民眾：對。醫師：阿身上有...
3,3,民眾：我自己昨天開始，一天吃一次一粒。醫師：一次一粒，減半量還好？民眾：阿可是……減到今天早...
4,4,醫師：喔，啊嘉明最近還好嗎？民眾：ＯＫ。醫師：啊今年的流感打了嗎？民眾：還沒。醫師：那我們今...
...,...,...
154,154,醫師：這個月還好嗎？民眾：就……醫師：這個月有去碰嗎？民眾：沒有，不去碰的時候就會覺得慾望非...
155,155,醫師：啊這個月還好嗎？民眾：還好啊。醫師：好，所以你反正天天吃啦齁？民眾：對。醫師：好，我們...
156,156,醫師：好那所以總之就是好，所以總之反正，總之就是……這這……就是反正你剩下的藥你就是……就是...
157,157,醫師：好，所以第一個月吃藥的狀況怎麼樣？民眾：嗯，吃藥狀況喔？醫師：嗯。民眾：就是吃了，一開...


In [4]:
entity_type_list = pd.read_csv('entity_type.csv')['entity_type'].tolist()
entity_type_list

['med_exam',
 'location',
 'time',
 'profession',
 'name',
 'money',
 'organization',
 'family',
 'ID',
 'clinical_event',
 'education',
 'contact',
 'others']

In [5]:
# contact_df = pd.read_csv('ground_truth/contact.csv')
# contact_list = contact_df['entity_text'].tolist()
# contact_list

In [6]:
def get_entity(article_id, text, entity_text, entity_type):
    output_str = str()
    source = '2-dict'
    for match in re.finditer(entity_text, text, re.IGNORECASE):
        output_str += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(article_id, match.start(), match.end(), match.group(), entity_type, source)
    return output_str

In [7]:
output_str = '{}\t{}\t{}\t{}\t{}\t{}\n'.format('article_id', 'start_position', 'end_position', 'entity_text', 'entity_type', 'source')

In [8]:
# ckip
entity_type_list = ['contact', 
                    'education', 
                    'ID', 
                    'family', 
                    'location', 
                    'money', 
                    'name', 
                    'organization', 
                    'profession', 
                    'time',
                   ]
for entity_type in entity_type_list:
    dictionary_path = 'ckip/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [9]:
# ground_truth
entity_type_list = ['clinical_event', 
                    'contact', 
                    'education', 
                    'family', 
                    'location', 
                    'organization', 
                    'others',
                   ]
for entity_type in entity_type_list:
    dictionary_path = 'ground_truth/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [10]:
# online
entity_type_list = ['contact', 
                   ]
for entity_type in entity_type_list:
    dictionary_path = 'online/{}.csv'.format(entity_type)
    entity_text_list = pd.read_csv(dictionary_path)['entity_text'].tolist()
    for entity_text in entity_text_list:
        for article_id, text in zip(df['article_id'], df['text']):
            output_str += get_entity(article_id, text, str(entity_text), entity_type)

In [11]:
# print(output_str)

In [12]:
with open('dictionary.tsv', 'w', encoding='utf-8') as f:
    f.write(output_str)

In [13]:
df = pd.read_csv('dictionary.tsv', sep='\t')
df = df.drop_duplicates()
df = df.sort_values(by=['article_id', 'start_position', 'end_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
528,0,36,38,年底,time,2-dict
557,0,113,115,今天,time,2-dict
99,0,227,229,麻豆,location,2-dict
100,0,237,239,麻豆,location,2-dict
920,0,295,297,今年,time,2-dict
...,...,...,...,...,...,...
1727,158,1113,1116,下個月,time,2-dict
918,158,1143,1145,今天,time,2-dict
511,158,1199,1201,小安,name,2-dict
919,158,1353,1355,今天,time,2-dict


In [14]:
df[df.duplicated(keep=False, subset=['article_id', 'start_position'])]

,article_id,start_position,end_position,entity_text,entity_type,source
560,3,43,45,今天,time,2-dict
1322,3,43,47,今天早上,time,2-dict
561,3,263,265,今天,time,2-dict
1323,3,263,267,今天早上,time,2-dict
2980,3,867,869,禮拜,time,2-dict
...,...,...,...,...,...,...
5210,157,3777,3781,九月十號,time,2-dict
1753,157,3785,3787,九月,time,2-dict
5211,157,3785,3789,九月十號,time,2-dict
3486,157,3822,3824,禮拜,time,2-dict


In [15]:
# 刪除小的 eg: 媽
df = df.drop_duplicates(keep='last', subset=['article_id', 'start_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
528,0,36,38,年底,time,2-dict
557,0,113,115,今天,time,2-dict
99,0,227,229,麻豆,location,2-dict
100,0,237,239,麻豆,location,2-dict
920,0,295,297,今年,time,2-dict
...,...,...,...,...,...,...
1727,158,1113,1116,下個月,time,2-dict
918,158,1143,1145,今天,time,2-dict
511,158,1199,1201,小安,name,2-dict
919,158,1353,1355,今天,time,2-dict


In [16]:
df[df.duplicated(keep=False, subset=['article_id', 'end_position'])]

,article_id,start_position,end_position,entity_text,entity_type,source
1322,3,43,47,今天早上,time,2-dict
1325,3,45,47,早上,time,2-dict
1323,3,263,267,今天早上,time,2-dict
1327,3,265,267,早上,time,2-dict
1564,5,726,732,10月25號,time,2-dict
...,...,...,...,...,...,...
2879,157,3734,3737,禮拜一,time,2-dict
1842,158,193,197,四個禮拜,time,2-dict
3487,158,195,197,禮拜,time,2-dict
5212,158,1082,1086,第三個月,time,2-dict


In [17]:
# 刪除小的 eg: 媽
df = df.drop_duplicates(keep='first', subset=['article_id', 'end_position'])
df

,article_id,start_position,end_position,entity_text,entity_type,source
528,0,36,38,年底,time,2-dict
557,0,113,115,今天,time,2-dict
99,0,227,229,麻豆,location,2-dict
100,0,237,239,麻豆,location,2-dict
920,0,295,297,今年,time,2-dict
...,...,...,...,...,...,...
1727,158,1113,1116,下個月,time,2-dict
918,158,1143,1145,今天,time,2-dict
511,158,1199,1201,小安,name,2-dict
919,158,1353,1355,今天,time,2-dict


In [18]:
df.to_csv('dictionary.tsv', sep='\t', index=False)